In [59]:
import pandas as pd
import numpy as np

df = pd.read_csv('dataset-umuai-2.csv',delimiter=';')
df

,id,url,title,cost,category,imageURL,description,prepTime,cookTime,totalTime,...,fibers,cholesterol,sodium,ingredients_measure,ingredients,instructions,bestRating,ratingCount,ratingValue,difficulty
0,0,https://ricette.giallozafferano.it/Pancakes-al...,Pancakes with dacere syrup,Molto basso,Dolci,https://www.giallozafferano.it/images/ricette/...,"Making pancakes at home is very simple, you ne...",PT15M,PT20M,PT35M,...,0.9,117.0,63.0,"[Butter 25g, Flour 00 125g, Medium Eggs 2, Fre...","[Butter, Flour 00, Eggs, Whole Milk, Baking Po...",Iniziamo la preparazione dei pancake fondendo ...,5.0,736.0,4.4,Molto facile
1,1,https://ricette.giallozafferano.it/Tiramisu.html,Tiramisu,Medio,Dolci,https://www.giallozafferano.it/images/ricette/...,Tiramisu is definitely one of the most delicio...,PT40M,PTM,PT40M,...,1.4,259.0,151.0,"[Savoiardi 300g, Fresh eggs (about 4 medium) 2...","[Savoiardi, Eggs, Mascarpone, Sugar, Coffee, C...",Per preparare il tiramisù cominciate dalle uov...,5.0,1055.0,4.2,Facile
2,2,https://ricette.giallozafferano.it/Crepes-dolc...,Sweet and salty crepes (basic recipe),Basso,Dolci,https://www.giallozafferano.it/images/ricette/...,Thanks to the basic dough of sweet and salty c...,PT10M,PT15M,PT25M,...,0.8,83.0,56.0,"[Medium eggs 3, Flour 00 250g, Whole milk 500m...","[Eggs, Flour 00, Whole Milk, Butter]",Per preparare le crepe dolci e salate iniziate...,5.0,381.0,4.2,Facile
3,3,https://ricette.giallozafferano.it/Spaghetti-a...,Spaghetti carbonara,Basso,Primi piatti,https://www.giallozafferano.it/images/ricette/...,The recipe for spaghetti carbonara is typical ...,PT15M,PT10M,PT25M,...,2.2,383.0,586.0,"[Spaghetti 320 g, Cheek 150g, Medium Egg Yolks...","[Spaghetti, Guanciale, Yolks, Roman Pecorino, ...",Per preparare gli spaghetti alla carbonara com...,5.0,854.0,4.2,Facile
4,4,https://ricette.giallozafferano.it/Impasto-per...,Pasta for pizza,Molto basso,Lievitati,https://www.giallozafferano.it/images/ricette/...,The dough for pizza is a basic preparation mad...,PT20M,PTM,PT20M,...,6.3,NaN,1966.0,"[Farina Manitoba 200g, Flour 00 300g, Water at...","[Farina Manitoba, Flour 00, Water, ""Extra Virg...",Per preparare la pasta per la pizza abbiamo sc...,5.0,367.0,4.2,Facile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4636,4636,https://ricette.giallozafferano.it/Sfoglie-all...,Olive leaf with topinambur and raspexdura,Basso,Antipasti,https://www.giallozafferano.it/images/ricette/...,"The olive sheets with topinambur and swagger, ...",PT20M,PT20M,PT40M,...,0.6,5.0,245.0,[Olivia&Marino's Workshop crispy baked leaves ...,"[Olivia&Marino's Workshop, Topinambur, Raspadu...",Per preparare le sfoglie alle olive con topina...,5.0,2.0,3.0,Molto facile
4637,4637,https://ricette.giallozafferano.it/Gelato-fior...,Fiordilatte ice cream,Basso,Dolci,https://www.giallozafferano.it/images/ricette/...,The fjordice ice cream is a summer season must...,PT10M,PT5M,PT15M,...,NaN,NaN,NaN,"[Whole milk 1 l, Fresh cold fridge cream 190g,...","[Whole milk, fresh liquid bread, sugar, vanill...",Per preparare il gelato fiordilatte come prima...,5.0,25.0,4.6,Facile
4638,4638,https://ricette.giallozafferano.it/Pasta-con-m...,Pasta with eggplant and Grana Padano cilad,NaN,Primi piatti,https://www.giallozafferano.it/images/ricette/...,Pasta with eggplant and grana Padano cilad is ...,PT15M,PT20M,PT35M,...,NaN,NaN,NaN,"[Aubergine 400g, Mezze Sleeves Rigate 320g, Gr...","[Aubergines, Half Sleeves Rigate, Grana Padano...",Per realizzare le mezze maniche con melanzane ...,5.0,1.0,5.0,Facile
4639,4639,https://ricette.giallozafferano.it/Cantucci-al...,Cantucci with Grana Padano and dried fruit,NaN,Antipasti,https://www.giallozafferano.it/images/ricette/...,The cantucci al Grana Padano and dried fruit a...,PT10M,PT40M,PT50M,...,NaN,NaN,NaN,"[Grana Padano DOP Reserve Over 20 months 60 g,...","[Grana Padano PDO, Albumi, Flour 0, Almonds, P...",Per preparare i cantucci al Grana Padano con f...,5.0,6.0,3.7,Facile


In [60]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords and create useful word vectors
tfidf = TfidfVectorizer(stop_words= 'english')

In [61]:
#Replace NaN with an empty string
df['ingredients'] = df['ingredients'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the ingredient feature
tfidf_matrix = tfidf.fit_transform(df['ingredients'])


In [62]:
#Output the shape of tfidf_matrix
print(tfidf_matrix.shape)

# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

(4641, 1542)


In [63]:
# Compute the cosine similarity matrix - this may take a few minutes
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse mapping of indices and ingredients
indices = pd.Series(df.index, index=df['title'])

cosine_sim

array([[1.        , 0.13664279, 0.42661041, ..., 0.        , 0.02923427,
        0.03116165],
       [0.13664279, 1.        , 0.08580206, ..., 0.        , 0.        ,
        0.        ],
       [0.42661041, 0.08580206, 1.        , ..., 0.        , 0.06852686,
        0.07304475],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.34656732,
        0.464826  ],
       [0.02923427, 0.        , 0.06852686, ..., 0.34656732, 1.        ,
        0.46208021],
       [0.03116165, 0.        , 0.07304475, ..., 0.464826  , 0.46208021,
        1.        ]])

In [64]:
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    #obtain index of the recipes that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all recipes with that recipe
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the recipes based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar recipes. Ignore the first recipes.
    sim_scores = sim_scores[1:11]

    # Get the recipe indices
    recipe_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar recipes
    return df['title'].iloc[recipe_indices]

In [65]:
def create_soup(x):
    tags = x['title'].lower().split(' ')
    tags.extend(x['ingredients'].lower().split())
    return " ".join(sorted(set(tags), key=tags.index))

In [66]:
df['soup'] = df.apply(create_soup, axis=1)

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a new TfidfVectorizer object and create vectors for the soup
count = TfidfVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [68]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity score (equivalent to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

count_matrix.shape

(4641, 3148)

In [69]:
cosine_sim3 = pd.DataFrame(cosine_sim2, index=df.index, columns=df.index)

cosine_sim3.shape

(4641, 4641)

In [70]:
cosine_sim3.head(10)

,0,1,2,3,4,5,6,7,8,9,...,4631,4632,4633,4634,4635,4636,4637,4638,4639,4640
0,1.000000,0.074283,0.080624,0.000000,0.037084,0.000000,0.103872,0.032596,0.000000,0.000000,...,0.000000,0.049950,0.000000,0.054887,0.012102,0.000000,0.054032,0.000000,0.012466,0.018240
1,0.074283,1.000000,0.022560,0.000000,0.000000,0.000000,0.047374,0.031111,0.000000,0.000000,...,0.000000,0.028542,0.000000,0.000000,0.000000,0.000000,0.023101,0.000000,0.000000,0.000000
2,0.080624,0.022560,1.000000,0.000000,0.042166,0.000000,0.100154,0.025274,0.000000,0.000000,...,0.000000,0.034249,0.000000,0.062408,0.013761,0.000000,0.043188,0.000000,0.014174,0.020739
3,0.000000,0.000000,0.000000,1.000000,0.009180,0.421682,0.000000,0.033826,0.655240,0.080089,...,0.026514,0.032345,0.023526,0.005982,0.006512,0.022801,0.000000,0.019436,0.000000,0.028276
4,0.037084,0.000000,0.042166,0.009180,1.000000,0.077591,0.046068,0.019761,0.010293,0.070055,...,0.053617,0.028929,0.047574,0.147128,0.238747,0.046107,0.052130,0.070569,0.072179,0.057179
5,0.000000,0.000000,0.000000,0.421682,0.077591,1.000000,0.000000,0.000000,0.487395,0.179492,...,0.070107,0.000000,0.081000,0.050562,0.013478,0.088689,0.000000,0.036340,0.042813,0.074766
6,0.103872,0.047374,0.100154,0.000000,0.046068,0.000000,1.000000,0.040492,0.000000,0.000000,...,0.000000,0.062050,0.000000,0.068183,0.015034,0.000000,0.067121,0.000000,0.015486,0.022658
7,0.032596,0.031111,0.025274,0.033826,0.019761,0.000000,0.040492,1.000000,0.000000,0.000000,...,0.000000,0.099187,0.000000,0.010553,0.026716,0.000000,0.203462,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.655240,0.010293,0.487395,0.000000,0.000000,1.000000,0.089797,...,0.041224,0.000000,0.036578,0.006707,0.007301,0.035450,0.000000,0.032553,0.000000,0.043963
9,0.000000,0.000000,0.000000,0.080089,0.070055,0.179492,0.000000,0.000000,0.089797,1.000000,...,0.043899,0.000000,0.058288,0.045651,0.006933,0.037751,0.000000,0.055854,0.044047,0.046817


In [71]:
def rename_columns(df, pattern):
    new_columns = [f"{pattern}{i}" for i in range(len(df.columns))]
    df.columns = new_columns

In [72]:
rename_columns(cosine_sim3, "sim")

In [73]:
cosine_sim3.head(10)

,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,...,sim4631,sim4632,sim4633,sim4634,sim4635,sim4636,sim4637,sim4638,sim4639,sim4640
0,1.000000,0.074283,0.080624,0.000000,0.037084,0.000000,0.103872,0.032596,0.000000,0.000000,...,0.000000,0.049950,0.000000,0.054887,0.012102,0.000000,0.054032,0.000000,0.012466,0.018240
1,0.074283,1.000000,0.022560,0.000000,0.000000,0.000000,0.047374,0.031111,0.000000,0.000000,...,0.000000,0.028542,0.000000,0.000000,0.000000,0.000000,0.023101,0.000000,0.000000,0.000000
2,0.080624,0.022560,1.000000,0.000000,0.042166,0.000000,0.100154,0.025274,0.000000,0.000000,...,0.000000,0.034249,0.000000,0.062408,0.013761,0.000000,0.043188,0.000000,0.014174,0.020739
3,0.000000,0.000000,0.000000,1.000000,0.009180,0.421682,0.000000,0.033826,0.655240,0.080089,...,0.026514,0.032345,0.023526,0.005982,0.006512,0.022801,0.000000,0.019436,0.000000,0.028276
4,0.037084,0.000000,0.042166,0.009180,1.000000,0.077591,0.046068,0.019761,0.010293,0.070055,...,0.053617,0.028929,0.047574,0.147128,0.238747,0.046107,0.052130,0.070569,0.072179,0.057179
5,0.000000,0.000000,0.000000,0.421682,0.077591,1.000000,0.000000,0.000000,0.487395,0.179492,...,0.070107,0.000000,0.081000,0.050562,0.013478,0.088689,0.000000,0.036340,0.042813,0.074766
6,0.103872,0.047374,0.100154,0.000000,0.046068,0.000000,1.000000,0.040492,0.000000,0.000000,...,0.000000,0.062050,0.000000,0.068183,0.015034,0.000000,0.067121,0.000000,0.015486,0.022658
7,0.032596,0.031111,0.025274,0.033826,0.019761,0.000000,0.040492,1.000000,0.000000,0.000000,...,0.000000,0.099187,0.000000,0.010553,0.026716,0.000000,0.203462,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.655240,0.010293,0.487395,0.000000,0.000000,1.000000,0.089797,...,0.041224,0.000000,0.036578,0.006707,0.007301,0.035450,0.000000,0.032553,0.000000,0.043963
9,0.000000,0.000000,0.000000,0.080089,0.070055,0.179492,0.000000,0.000000,0.089797,1.000000,...,0.043899,0.000000,0.058288,0.045651,0.006933,0.037751,0.000000,0.055854,0.044047,0.046817


In [74]:
#create a column with index numbers as a ID.
#A better is probably to create ID numbers in the original file as well as and to sort the original df on those ID numbers
cosine_sim3['id'] = df.index
cosine_sim3 = cosine_sim3[['id'] + [col for col in cosine_sim3.columns if col != 'id']]
cosine_sim3.head(5)

,id,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,...,sim4631,sim4632,sim4633,sim4634,sim4635,sim4636,sim4637,sim4638,sim4639,sim4640
0,0,1.000000,0.074283,0.080624,0.00000,0.037084,0.000000,0.103872,0.032596,0.000000,...,0.000000,0.049950,0.000000,0.054887,0.012102,0.000000,0.054032,0.000000,0.012466,0.018240
1,1,0.074283,1.000000,0.022560,0.00000,0.000000,0.000000,0.047374,0.031111,0.000000,...,0.000000,0.028542,0.000000,0.000000,0.000000,0.000000,0.023101,0.000000,0.000000,0.000000
2,2,0.080624,0.022560,1.000000,0.00000,0.042166,0.000000,0.100154,0.025274,0.000000,...,0.000000,0.034249,0.000000,0.062408,0.013761,0.000000,0.043188,0.000000,0.014174,0.020739
3,3,0.000000,0.000000,0.000000,1.00000,0.009180,0.421682,0.000000,0.033826,0.655240,...,0.026514,0.032345,0.023526,0.005982,0.006512,0.022801,0.000000,0.019436,0.000000,0.028276
4,4,0.037084,0.000000,0.042166,0.00918,1.000000,0.077591,0.046068,0.019761,0.010293,...,0.053617,0.028929,0.047574,0.147128,0.238747,0.046107,0.052130,0.070569,0.072179,0.057179


In [18]:
#the recommender with input
indices2 = pd.Series(df.index, index=df['title'])

print(content_recommender(input("What do you want to eat?"), cosine_sim2, df, indices2))

What do you want to eat?Rice salad
96       Cold rice with tuna, zucchini and lemon
3743                         Rice and tuna cakes
1237    Wild and wild rice salad with vegetables
749                    Peppers stuffed with rice
3314                          Tartine with jelly
2243              Lemons stuffed with tuna cream
199                                Russian salad
1334               Rice salad with ham and melon
3099                         Easter stuffed eggs
29                                 Rice arancini
Name: title, dtype: object
